#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (`model_name="sentence-transformers/all-mpnet-base-v2"`)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

# ADD HERE YOUR CODE
embedding_model = HuggingFaceEmbeddings (model_name="sentence-transformers/all-mpnet-base-v2")

/workspaces/TeamMaxiKing/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/workspaces/TeamMaxiKing/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
text = "This is a test document."

# ADD HERE YOUR CODE
# Perform vector search
query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 768
[-0.04895179718732834, -0.03986203297972679, -0.021562788635492325, 0.00990856159478426, -0.038103923201560974, 0.01268438808619976, 0.04349448159337044, 0.07183388620615005, 0.009748609736561775, -0.006987091153860092]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: `PyPDFLoader`
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = PyPDFLoader(pdf_doc)

# Load and split documents in chunks
# ADD HERE YOUR CODE
pages_chunked = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],  # Specified splitting characters
    chunk_size=1000,  # Adjust to your desired chunk size
    chunk_overlap=100  # Overlap between chunks, useful for context
)
pages = loader.load()  # Load all pages from the PDF
pages_chunked = text_splitter.split_documents(pages)

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

pages_chunked_cleaned = [clean_text(chunk.page_content) for chunk in pages_chunked]

print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [4]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.' metadata={'source

In [5]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 1272


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)


In [6]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection

collection_name = "AI_Book"

collection = client.get_or_create_collection(collection_name)

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=embedding_model
)

In [7]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

# ADD HERE YOUR CODE
vector_db_from_client.add_texts(texts=pages_chunked_cleaned[:50], ids=uuids)


['88e5df45-3147-4e74-8557-7066f16fb1b6',
 'bf7baa9d-7834-40a8-9a15-c7edcc150c87',
 '214533c9-6bac-4280-b7a4-4f407de1a133',
 '57fbe9f5-aa60-4f11-9f08-82e04edda58b',
 '1a0797f0-7034-42b0-abd7-905aefc0df23',
 '8a00f77f-ee45-483b-907e-ffda154cfa33',
 '9990c6ce-0e17-4702-8c35-f393c35b2d0e',
 'e7937e69-a93e-461d-a56f-e4f146b0dd69',
 '4d5ee7de-20f9-47e4-9cf5-e1ee85d8f350',
 '1c1d2066-e7b4-4c69-9782-d177fad2eff3',
 '7776410c-acf2-4218-ae9b-26e44fee4ed7',
 '97409fec-4eda-4cbd-8103-8d5f2d9a3ca7',
 '63a0f92d-0661-47df-b59c-e5c5c117b8bf',
 '67f3b3a7-86d6-4ac5-87a7-02ea6b397741',
 '7f22ec34-ae53-4493-9f26-9412a39d9ee9',
 'f73cf84a-c4fc-4f87-a886-b2055d230176',
 'e01d0f2f-b327-496b-8599-ebf940e73c66',
 '28d9c2ee-6598-48bb-a657-79634f1d0278',
 '435eadd3-698b-49d1-86f8-20709bea5c80',
 '8e009126-ea01-4703-82cc-865047ac6c55',
 '2e5e1f06-0db8-4027-a34c-ab6ede34b30a',
 '81bdc0e6-9783-4a3c-9090-a5bd95ee9293',
 'c6bc8e22-74c3-4089-878a-5d21e39be0b0',
 '32b56682-a6b6-49e2-b2f7-f147dcbec131',
 'e13525f2-f701-

In [8]:
client.count_collections()

1

In [9]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents

**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)


In [10]:
search_query = "Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search(query=search_query, k=3)

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

The most common learning algorithms: Linear and Polynomial Regression,
Logistic Regression, k-Nearest Neighbors, Support Vector Machines, Decision
Trees, Random Forests, and Ensemble methods.
xiv | Preface
{}


The most common learning algorithms: Linear and Polynomial Regression,
Logistic Regression, k-Nearest Neighbors, Support Vector Machines, Decision
Trees, Random Forests, and Ensemble methods.
xiv | Preface
{}


The most common learning algorithms: Linear and Polynomial Regression,
Logistic Regression, k-Nearest Neighbors, Support Vector Machines, Decision
Trees, Random Forests, and Ensemble methods.
xiv | Preface
{}


